In [1]:
# https://mne.tools/stable/auto_tutorials/forward/30_forward.html
# use anaconda 3 mne enviroment

import mne
from mne.datasets import sample
data_path = sample.data_path()

# the raw file containing the channel location + types
raw_fname = '../CAMCAN_MEG_100/CC110033/task/task_raw.fif'
# The paths to Freesurfer reconstructions
subjects_dir = data_path / 'subjects'
subject = 'fsaverage' # change it to use freesurfer's bem

In [2]:
trans = './test_scripts/CAMCAN_fsaverage_trans.fif'


In [3]:
info = mne.io.read_info(raw_fname)

    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle


In [4]:
info.keys()

dict_keys(['file_id', 'events', 'hpi_results', 'hpi_meas', 'subject_info', 'device_info', 'helium_info', 'hpi_subsystem', 'proc_history', 'meas_id', 'experimenter', 'description', 'proj_id', 'proj_name', 'meas_date', 'utc_offset', 'sfreq', 'highpass', 'lowpass', 'line_freq', 'gantry_angle', 'chs', 'dev_head_t', 'ctf_head_t', 'dev_ctf_t', 'dig', 'bads', 'ch_names', 'nchan', 'projs', 'comps', 'acq_pars', 'acq_stim', 'custom_ref_applied', 'xplotter_layout', 'kit_system_id'])

In [4]:
# Compute Source Space (surface)
# https://mne.tools/0.19/generated/mne.setup_source_space.html
# The spacing to use. Can be 'ico#' for a recursively subdivided icosahedron, 
# 'oct#' for a recursively subdivided octahedron, 
# 'all' for all points, or an integer to use appoximate distance-based spacing (in mm).
src = mne.setup_source_space(subject, spacing='ico3', add_dist='patch',
                             subjects_dir=subjects_dir)
# oct8 takes 2.5 min produces 65538 source
# ico7 = 'all'


Setting up the source space with the following parameters:

SUBJECTS_DIR = /home/zhibinz2/mne_data/MNE-sample-data/subjects
Subject      = fsaverage
Surface      = white
Icosahedron subdivision grade 3

>>> 1. Creating the source space...

Doing the icosahedral vertex picking...
Loading /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/surf/lh.white...
Mapping lh fsaverage -> ico (3) ...
    Triangle neighbors and vertex normals...
Loading geometry from /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 642/163842 selected to source space (ico = 3)

Loading /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/surf/rh.white...
Mapping rh fsaverage -> ico (3) ...
    Triangle neighbors and vertex normals...
Loading geometry from /home/zhibinz2/mne_data/MNE-sample-data/subjects/fsaverage/surf/rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.whi

In [5]:
# compute forward solution
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject='fsaverage', ico=3,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 3th subdivision of an icosahedron (n_tri: 20480 -> 1280)
inner skull CM is  -0.53 -21.10   6.21 mm
Surfaces passed the basic topology checks.
Complete.

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner skull (642) -> inner skull (642) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.


In [6]:
fwd = mne.make_forward_solution(raw_fname, trans=trans, src=src, bem=bem,
                                meg=True, eeg=False, mindist=5.0, n_jobs=2,
                                verbose=True)
print(fwd)

Source space          : <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=642>, <surface (rh), n_vertices=163842, n_used=642>] MRI (surface RAS) coords, subject 'fsaverage', ~30.2 MB>
MRI -> head transform : ./test_scripts/CAMCAN_fsaverage_trans.fif
Measurement data      : task_raw.fif
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 1284 active source locations



Coordinate transformation: MRI (surface RAS) -> head
     0.999927 -0.009207 -0.007775      -4.28 mm
     0.009519  0.999110  0.041099      24.90 mm
     0.007390 -0.041170  0.999125      31.33 mm
     0.000000  0.000000  0.000000       1.00

Read 306 MEG channels from info
105 coil definitions read
Coordinate transformation: MEG device -> head
     0.998429 -0.050574  0.024122       2.53 mm
     0.052468  0.994938 -0.085709     -15.30 mm
    -0.019665  0.086840  0.996028      39.95 mm
     0.000000  0.000000  0.000000       1.00
MEG coil definitions created in head coordinates.
Source spaces are now in head coordinates.

Employing the head->MRI coordinate transform with the BEM model.
BEM model instance of ConductorModel is now set up

Source spaces are in head coordinates.
Checking that the sources are inside the surface and at least    5.0 mm away (will take a few...)
Checking surface interior status for 642 points...
    Found 157/642 points inside  an interior sphere of radius   

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


    Found   0/485 points outside using solid angles
    Total 642/642 points inside the surface
Interior check completed in 5604.1 ms
Checking surface interior status for 642 points...
    Found 147/642 points inside  an interior sphere of radius   48.4 mm
    Found   0/642 points outside an exterior sphere of radius   97.8 mm
    Found   0/495 points outside using surface Qhull
    Found   0/495 points outside using solid angles
    Total 642/642 points inside the surface
Interior check completed in 100.7 ms

Checking surface interior status for 306 points...
    Found   0/306 points inside  an interior sphere of radius   48.4 mm
    Found 306/306 points outside an exterior sphere of radius   97.8 mm
    Found   0/  0 points outside using surface Qhull
    Found   0/  0 points outside using solid angles
    Total 0/306 points inside the surface
Interior check completed in 43.6 ms

Composing the field computation matrix...


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    5.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jo

Computing MEG at 1284 source locations (free orientations)...


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.



Finished.
<Forward | MEG channels: 306 | EEG channels: 0 | Source space: Surface with 1284 vertices | Source orientation: Free>


[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.1s finished


In [7]:
print(f'Before: {src}')
print(f'After:  {fwd["src"]}')

Before: <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=642>, <surface (rh), n_vertices=163842, n_used=642>] MRI (surface RAS) coords, subject 'fsaverage', ~30.2 MB>
After:  <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=642>, <surface (rh), n_vertices=163842, n_used=642>] head coords, subject 'fsaverage', ~30.2 MB>


In [8]:
# leadfield = fwd['sol']['data']
# print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 306 sensors x 3852 dipoles


In [9]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Leadfield size : 306 sensors x 1284 dipoles


In [10]:
from hdf5storage import loadmat, savemat 
outdict=dict()
outdict['leadfield']=leadfield
outdict['source_nn']=fwd_fixed['source_nn']
outdict['source_rr']=fwd_fixed['source_rr']
savemat('leadfield_nn_rr',outdict)

In [14]:
mne.write_forward_solution('surface_simple_CAM_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=True, verbose=None)

    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


/tmp/ipykernel_92931/3611321024.py:1: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  mne.write_forward_solution('surface_simple_CAM_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=True, verbose=None)


In [19]:
# forward = mne.read_forward_solution('simple_fsaverage_fixed_fwd.fif',verbose=None)

Reading forward solution from /home/zhibinz2/Documents/GitHub/MEG_EEG_Source_Localization/simple_fsaverage_fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (516 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
